# DeepEval: LLM을 위한 유닛 테스트 도구
- https://discuss.pytorch.kr/t/deepeval-llm/2557
- https://www.reddit.com/r/Python/comments/1i2kafp/deepeval_the_opensource_llm_evaluation_framework/
- https://deepeval.com/integrations/frameworks/langgraph

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## End-to-End

In [3]:
import asyncio

from langchain.agents import create_agent
from deepeval.integrations.langchain import CallbackHandler
from deepeval.metrics import TaskCompletionMetric
from deepeval.dataset import Golden, EvaluationDataset

def get_weather(city: str) -> str:
    """Returns the weather in a city"""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model="openai:gpt-4.1-mini",  
    tools=[get_weather],  
    system_prompt="You are a helpful assistant",
)

dataset = EvaluationDataset(goldens=[
    Golden(input="What is the weather in Bogotá, Colombia?"),
    Golden(input="What is the weather in Paris, France?"),
])
 
for golden in dataset.evals_iterator():
    agent.invoke(
        input={"messages": [{"role": "user", "content": golden.input}]},
        config={"callbacks": [CallbackHandler(metrics=[TaskCompletionMetric()])]}
    )

Output()



Metrics Summary


For test case:

  - input: What is the weather in Paris, France?
  - actual output: {'messages': [HumanMessage(content='What is the weather in Paris, France?', additional_kwargs={}, response_metadata={}, id='c74ed893-1bd8-428c-8d24-9938634d7721'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 59, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_9766e549b2', 'id': 'chatcmpl-CjQSFuZwi32sFyzlIPDMO83MlTDTl', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019aeec5-16a9-70f3-a239-7a96cdb64692-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Paris'}, 'id': 'call_N43w7

⚠ WARNING: No hyperparameters logged.
» ]8;id=533338;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 12.87s | token cost: None USD)
» Test Results (2 total tests):
   » Pass Rate: 50.0% | Passed: 1 | Failed: 1

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

## Component Eval

In [6]:
# from langchain_openai import ChatOpenAI
# from deepeval.metrics import AnswerRelevancyMetric
# from rich import print as rprint


# llm = ChatOpenAI(
#     model="gpt-4o-mini", 
#     metadata={"metric": [AnswerRelevancyMetric()]}
# ).bind_tools([get_weather])

# agent = create_agent(
#     model="openai:gpt-4.1-mini",  
#     tools=[get_weather],  
#     system_prompt="You are a helpful assistant",
# )

# dataset = EvaluationDataset(goldens=[
#     Golden(input="What is the weather in Bogotá, Colombia?"),
#     Golden(input="What is the weather in Paris, France?"),
# ])
 
# for golden in dataset.evals_iterator:
#     result = agent.invoke(
#         input={"messages": [{"role": "user", "content": golden.input}]},
#         config={"callbacks": [CallbackHandler(metrics=[TaskCompletionMetric()])]}
#     )
#     rprint(result)

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
# from langchain_core.tools import tool
from deepeval.integrations.langchain import tool
from deepeval.metrics import AnswerRelevancyMetric

@tool(metric=[AnswerRelevancyMetric()])
def get_weather(location: str) -> str:
    """Get the current weather in a location."""
    return f"It's always sunny in {location}!"

agent = create_agent(
    model="openai:gpt-4.1-mini",  
    tools=[get_weather],  
    system_prompt="You are a helpful assistant",
)

dataset = EvaluationDataset(goldens=[
    Golden(input="What is the weather in Bogotá, Colombia?"),
    Golden(input="What is the weather in Paris, France?"),
])
 
for golden in dataset.evals_iterator():
    agent.invoke(
        input={"messages": [{"role": "user", "content": golden.input}]},
        config={"callbacks": [CallbackHandler(metrics=[TaskCompletionMetric()])]}
    )

TypeError: tool() got an unexpected keyword argument 'metric'

In [1]:
from openai import AsyncOpenAI
from ragas.llms import llm_factory
from ragas.metrics.collections import Faithfulness

# Setup LLM
client = AsyncOpenAI()
llm = llm_factory("gpt-4o-mini", client=client)

# Create metric
scorer = Faithfulness(llm=llm)

# Evaluate
result = await scorer.ascore(
    user_input="When was the first super bowl?",
    response="The first superbowl was held on Jan 15, 1967",
    retrieved_contexts=[
        "The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."
    ]
)
print(f"Faithfulness Score: {result.value}")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable